### Download NDVI from Sentinel and Landsat using GEE

Search for all scenes within a collection that are: 

- within a date range

- intersect a polygon 

- have <=X % cloud cover 

- calculate ndvi for each scene
    
Then save those scenes to my google drive 

In [9]:
import ee
import geemap 
import os
import geemap
ee.Initialize()


Helpful for pulling out metadata of scenes in gee python api: https://samapriya.github.io/gee-py/projects/collection_meta/

Landsat Dates: 

Landsat 5: 1984–2012

Landsat 7: 1999–Present

Landsat 8: 2013–Present

In [10]:
# https://gis.stackexchange.com/questions/274048/apply-cloud-mask-to-landsat-imagery-in-google-earth-engine-python-api
def getQABits(image, start, end, mascara):
    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start,end+1):
        pattern += 2**i
    # Return a single band image of the extracted QA bits, giving the band a new name.
    return image.select([0], [mascara]).bitwiseAnd(pattern).rightShift(start)

#A function to mask out cloudy pixels.
def maskQuality_l8(image):
    # Select the QA band.
    QA = image.select('QA_PIXEL')
    # Get the internal_cloud_algorithm_flag bit.
    cloud_shadow = getQABits(QA,4,4,'cloud_shadow')
    cloud = getQABits(QA,3,3,'cloud')
    cirrus = getQABits(QA,2,2,'cirrus_detected')
    #Return an image masking out cloudy areas.
    return image.updateMask(cloud_shadow.eq(0)).updateMask(cloud.eq(0).updateMask(cirrus.eq(0)))

def maskQuality_l57(image):
    # Select the QA band.
    QA = image.select('QA_PIXEL')
    # Get the internal_cloud_algorithm_flag bit.
    cloud_shadow = getQABits(QA,4,4,'cloud_shadow')
    cloud = getQABits(QA,3,3,'cloud')
    #Return an image masking out cloudy areas.
    return image.updateMask(cloud_shadow.eq(0)).updateMask(cloud.eq(0))

def clip(ROI):
    def wrap(image):
        clipimage = image.clip(ROI)
        return clipimage
    return wrap

# A function to calculate NDVI 
def calcNDVI(NIR, RED):
    def wrap(image):
        ndvi = image.normalizedDifference([NIR, RED]).rename('NDVI')
        return image.addBands(ndvi)
    return wrap

def CalcLandsat8NDVI(COLLECTION, STARTMONTH, ENDMONTH, ROI, MAXCLOUD, NIR, RED):
    # filter by the conditions, mask out the clouds, and calculate the ndvi 
    collection = ee.ImageCollection(COLLECTION).filter(ee.Filter.calendarRange(STARTMONTH, ENDMONTH, 'month')).filterBounds(ROI).filter(ee.Filter.lt('CLOUD_COVER', MAXCLOUD)).map(maskQuality_l8).map(calcNDVI(NIR, RED))
    count = collection.size()
    print('Count: ', str(count.getInfo())+'\n')
    return collection

def CalcLandsat57NDVI(COLLECTION, STARTMONTH, ENDMONTH, ROI, MAXCLOUD, NIR, RED):
    # filter by the conditions, mask out the clouds, and calculate the ndvi 
    collection = ee.ImageCollection(COLLECTION).filter(ee.Filter.calendarRange(STARTMONTH, ENDMONTH, 'month')).filterBounds(ROI).filter(ee.Filter.lt('CLOUD_COVER', MAXCLOUD)).map(maskQuality_l57).map(calcNDVI(NIR, RED))
    count = collection.size()
    print('Count: ', str(count.getInfo())+'\n')
    return collection

def CalcLandsat8NDVI_ingeom(COLLECTION, STARTMONTH, ENDMONTH, ROI, MAXCLOUD, NIR, RED):
    # filter by the conditions, clip to the ROI, mask out the clouds, and calculate the ndvi 
    collection = ee.ImageCollection(COLLECTION).filter(ee.Filter.calendarRange(STARTMONTH, ENDMONTH, 'month')) \
        .filterBounds(ROI) \
        .map(clip(ROI)) \
        .filter(ee.Filter.lt('CLOUD_COVER', MAXCLOUD)) \
        .map(maskQuality_l8) \
        .map(calcNDVI(NIR, RED))
    
    count = collection.size()
    print('Count: ', str(count.getInfo())+'\n')
    return collection

def CalcLandsat57NDVI_ingeom(COLLECTION, STARTMONTH, ENDMONTH, ROI, MAXCLOUD, NIR, RED):
    # filter by the conditions, clip to the ROI, mask out the clouds, and calculate the ndvi 
    collection = ee.ImageCollection(COLLECTION).filter(ee.Filter.calendarRange(STARTMONTH, ENDMONTH, 'month')) \
        .filterBounds(ROI) \
        .map(clip(ROI)) \
        .filter(ee.Filter.lt('CLOUD_COVER', MAXCLOUD)) \
        .map(maskQuality_l57) \
        .map(calcNDVI(NIR, RED))
    
    count = collection.size()
    print('Count: ', str(count.getInfo())+'\n')
    return collection


In [11]:
Landsat5 = 'LANDSAT/LT05/C02/T1_L2'
Landsat7 = 'LANDSAT/LE07/C02/T1_L2'
Landsat8 = 'LANDSAT/LC08/C02/T1_L2'
#ROI = ee.FeatureCollection('users/kmcquil/SBR_XL_simplified').geometry()
ROI = ee.FeatureCollection('users/kmcquil/SBR_XL_simplified').geometry()

# landsat 8 cloud QA bits accurate ---- idk if it will be for the others 
l8_ndvi = CalcLandsat8NDVI(Landsat8, 6, 8, ROI, 20, 'SR_B5', 'SR_B4')
l7_ndvi = CalcLandsat57NDVI(Landsat7, 6, 8, ROI, 20, 'SR_B4', 'SR_B3' )
l5_ndvi=CalcLandsat57NDVI(Landsat5, 6, 8, ROI, 20, 'SR_B4', 'SR_B3')    

Count:  175

Count:  489

Count:  616



In [12]:
# export the collection 
geemap.ee_export_image_collection_to_drive(ee_object=l8_ndvi.select(['NDVI']), 
folder='Landsat_images_exported_gee', 
scale = 30,
region=ROI
)

geemap.ee_export_image_collection_to_drive(ee_object=l7_ndvi.select(['NDVI']), 
folder='Landsat_images_exported_gee', 
scale = 30,
region=ROI
)

geemap.ee_export_image_collection_to_drive(ee_object=l5_ndvi.select(['NDVI']), 
folder='Landsat_images_exported_gee', 
scale = 30,
region=ROI
)

Total number of images: 175

Exporting LC08_016034_20130615 ...
Exporting LC08_016034_20140602 ...
Exporting LC08_016034_20140618 ...
Exporting LC08_016034_20140704 ...
Exporting LC08_016034_20150707 ...
Exporting LC08_016034_20150824 ...
Exporting LC08_016034_20160607 ...
Exporting LC08_016034_20160826 ...
Exporting LC08_016034_20170610 ...
Exporting LC08_016034_20170626 ...
Exporting LC08_016034_20180629 ...
Exporting LC08_016034_20190702 ...
Exporting LC08_016034_20190819 ...
Exporting LC08_016034_20200602 ...
Exporting LC08_016034_20200704 ...
Exporting LC08_016034_20200720 ...
Exporting LC08_016034_20200805 ...
Exporting LC08_016034_20210605 ...
Exporting LC08_016034_20210621 ...
Exporting LC08_016034_20210707 ...
Exporting LC08_016034_20210808 ...
Exporting LC08_016034_20210824 ...
Exporting LC08_016035_20130615 ...
Exporting LC08_016035_20140602 ...
Exporting LC08_016035_20140618 ...
Exporting LC08_016035_20140704 ...
Exporting LC08_016035_20150621 ...
Exporting LC08_016035_2015

In [13]:
# Find cloud free sentinel-2 scenes, calculate NDVI, and download 

# the sentinel-2 page on GEE documentation links to a specific tutorial to remove clouds so that is the one I will use 
#https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless

def get_s2_sr_cld_col_mo(aoi, start_date, end_date, start_month, end_month):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .map(clip(aoi))
        .filterDate(start_date, end_date)
        .filter(ee.Filter.calendarRange(start_month, end_month, 'month'))
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .map(clip(aoi))
        .filter(ee.Filter.calendarRange(start_month, end_month, 'month')))

    count = s2_sr_col.size()
    print('Count: ', str(count.getInfo())+'\n')

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))
         

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))


def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))
    

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    # this line is if we want to look at the map with all of the layers 
    return img_cloud_shadow.addBands(is_cld_shdw)
    # this line is if we want to only include the final mask along with the original bands
    #return img.addBands(is_cld_shdw)

def maskClouds(img):
    QA = img.select('cloudmask')
    return img.updateMask(QA.eq(0))

In [14]:
ROI = ee.FeatureCollection('users/kmcquil/SBR_XL_simplified').geometry()
START_DATE = '2017-06-01'
END_DATE = '2021-08-30'
START_MONTH = 6
END_MONTH = 7
CLOUD_FILTER = 20
CLD_PRB_THRESH = 30
NIR_DRK_THRESH = 0.3
CLD_PRJ_DIST = 3
BUFFER = 50

s2_sr_cld_col_eval = get_s2_sr_cld_col_mo(ROI, START_DATE, END_DATE, START_MONTH, END_MONTH)
s2_sr_cld_col_eval_disp = s2_sr_cld_col_eval.map(add_cld_shdw_mask).map(maskClouds).map(calcNDVI('B8', 'B4'))

Count:  410



In [15]:
geemap.ee_export_image_collection_to_drive(ee_object=s2_sr_cld_col_eval_disp.select(['NDVI']), 
folder='Sentinel_images_exported_gee', 
scale = 10,
region=ROI
)

Total number of images: 410

Exporting 20190601T160901_20190601T161838_T16SGC ...
Exporting 20190601T160901_20190601T161838_T16SGE ...
Exporting 20190601T160901_20190601T161838_T17SKA ...
Exporting 20190601T160901_20190601T161838_T17SKT ...
Exporting 20190601T160901_20190601T161838_T17SKV ...
Exporting 20190601T160901_20190601T161838_T17SLA ...
Exporting 20190601T160901_20190601T161838_T17SLU ...
Exporting 20190601T160901_20190601T161838_T17SLV ...
Exporting 20190601T160901_20190601T161838_T17SMB ...
Exporting 20190601T160901_20190601T161838_T17SMU ...
Exporting 20190601T160901_20190601T161838_T17SMV ...
Exporting 20190601T160901_20190601T161838_T17SNB ...
Exporting 20190601T160901_20190601T161838_T17SNV ...
Exporting 20190603T155829_20190603T160233_T17SNB ...
Exporting 20190603T155829_20190603T160233_T17SPB ...
Exporting 20190604T161901_20190604T162855_T16SFE ...
Exporting 20190611T160901_20190611T161230_T17SKA ...
Exporting 20190611T160901_20190611T161230_T17SLA ...
Exporting 2019061